<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Internal reference output test
### Description
This script tests the REF OUT capability of the proteus module for the internal reference frequency of 100MHZ.Oscilloscope will measure the square signal of 100MHz available on REF OUT pin of the Proteus module.

### Equipment
- Proteus P9484M 
- Oscilloscope (MSO9254A)
- 1 SMP to BNC cable

### set up:
* Connect REF OUT of Proteus to the channel1 of the Oscilloscope.


### Proteus version: 
Aurora



In [9]:
 #Pre-requisite for the script 
ip_address = '192.90.70.22' #input('Please enter ip address for the device') 
test_success = True
channel_on_scope = 1 # ref out of the proteus is connected to channel3 on the oscilloscope

In [10]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

# Connect to instrument
inst = connect(ip_address)

# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{0}" after writing binary values'.format(resp))

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
DAC 16 bits


In [11]:
# maximum and minimum reference oscillator input
resp = inst.send_scpi_query(":ROSC:FREQ?")
print("Reference oscillator freq is changed to: " + resp)

resp = inst.send_scpi_query(":ROSC:FREQ?MAX")
print('Maximum reference value = ',resp)

resp = inst.send_scpi_query(":ROSC:FREQ?MIN")
print('Minimum reference value = ',resp)
resp = inst.send_scpi_query(":ROSC:FREQ?DEF")
print('Default reference value = ',resp)

Reference oscillator freq is changed to: 100M
Maximum reference value =  100M
Minimum reference value =  10M
Default reference value =  100M


In [12]:
vertical_scale_per_divison = 100e-3
horizontal_scale_per_divison = 5e-9
scope_time_scale = 2e-9
riseFall_time = 60e-12
scope_addr2= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    ## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('Connect to ',idn.rstrip('\n'))

Connect to  KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [13]:
#External reference input of 100M test 
inst.send_scpi_cmd(':ROSC:SOUR INT') # Internal reference source selected

scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(channel_on_scope,horizontal_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(channel_on_scope,vertical_scale_per_divison))
# scope.write('AUTOscale')
# time.sleep(5)
# scope.write('*OPC')
# scope.write('*CLS;:DISPlay:CGRade:LEVels')
scope.write(':MEASure:CLEar')
scope.write(':MEASure:FREQuency CHANnel{0}'.format(channel_on_scope))
time.sleep(1)
scope.write(':MEASure:RESults?')
result = scope.read()
frequency = float(result.split(',')[2])/1e6
#print('Frequency : {}MHz'.format(float(frequency)))
#     #print("Ocsilloscope frequency should {}MHz".format(cfreq ))
difference =abs(100 -frequency)
#print('Difference between measured frequency = {}MHz'.format(abs(100 -frequency)))
if difference < (2.5):
    print('Test pass for internal reference input 100MHz')
else:
    print('Test FAIL for internal reference input 100MHz')
    test_success = False

Test pass for internal reference input 100MHz


In [14]:
if(test_success):
    print('Test successed')
else:
     print('Test failed')
# Disconnect
disconnect()
print('Proteus disconnected successfully')

Test successed
Proteus disconnected successfully
